In [ ]:
from sklearn.datasets import fetch_openml   # Fetch dataset from openml

In [ ]:
mnist = fetch_openml("mnist_784", version=1, cache=True, as_frame=False)# mnist 데이터 가져오기. 28 * 28 = 784개의 속성 

In [ ]:
# 학습데이터, 타겟 데이터 
X = mnist.data
y = mnist.target

In [ ]:
import torch   # pytorch 
from torch.utils.data import TensorDataset, DataLoader  # data loading utility
from sklearn.model_selection import train_test_split  # Split arrays or matrices into random train and test subsets

In [ ]:
# proportion of the dataset to include in the train split : 1/7 , random_state: None 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=1/7, random_state=0)
# multi-dimensional matrix containing elements of a single data type
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
# 64-bit integer (signed)
y_train = torch.LongTensor(list(map(int, y_train)))
y_test = torch.LongTensor(list(map(int, y_test)))

In [ ]:
import torch.nn as nn  # neural network 
import torch.nn.functional as F # dropout, relu 
from torch import optim # optimization algorithms
from torch.autograd import Variable # automatic differentiation

In [ ]:
X_train = X_train.view(-1, 1, 28,28).float() # reshape
X_test = X_test.view(-1, 1, 28,28).float()  # reshape 
print(X_train.shape)
print(X_test.shape)

torch.Size([60000, 1, 28, 28])
torch.Size([10000, 1, 28, 28])


In [ ]:
train = TensorDataset(X_train, y_train)  # training data
test = TensorDataset(X_test, y_test)  # test data 
BATCH_SIZE = 32
loader_train = DataLoader(train, batch_size= BATCH_SIZE, shuffle=False) # batch size = 32, no shuffle 
loader_test = DataLoader(test, batch_size= BATCH_SIZE, shuffle=False)  # batch size = 32, no shuffle 


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # nn.Conv2d의 첫 두 파라미터는 입력 채널수(in_channels)와 출력 채널수(out_channels)
        self.conv1 = nn.Conv2d(1,32,kernel_size=5) #32개의 특징맵  kernel size : 5 x 5 
        self.conv2 = nn.Conv2d(32,32,kernel_size=5) #32개의 특징맵에서 32개의  특징맵. 
        self.conv3 = nn.Conv2d(32,64,kernel_size=5)  #32개에서 64개의 특징맵 
        self.fc1 = nn.Linear(3 * 3 * 64, 256) # linear transformation 
        self.fc2 = nn.Linear(256, 10)  # linear transformation 

        self.loss_fn = nn.CrossEntropyLoss() # loss function. CrossEntropy
        self.optimizer = optim.Adam(self.parameters(), lr=0.01) # adam optimizer. learning rate : 0.01

    def forward(self, x):  # forward function 
        x = F.relu(self.conv1(x)) # reLU
        x = F.relu(F.max_pool2d(self.conv2(x), 2))  # reLU
        x = F.dropout(x, p=0.5, training=self.training) # dropout 
        x = F.relu(F.max_pool2d(self.conv3(x), 2))  # reLU
        x = F.dropout(x, p=0.5, training=self.training)   # dropout 
        x = x.view(-1,3*3*64) # reshape. (-1은 남는차원 모두)
        x = F.relu(self.fc1(x))   # reLU
        x = F.dropout(x, training=self.training)  # dropout 
        x = self.fc2(x)  # 0부터 9까지 레이블을 갖는 10개의 출력값을 가지는 신경망
        return F.log_softmax(x, dim=1) # return log softmax 

    

In [ ]:
def fit(model, loader_train):
    optimizer = torch.optim.Adam(model.parameters()) # use Adam optimizer 
    error = nn.CrossEntropyLoss() # use Cross entropy 
    EPOCHS = 1 # 에포크 
    model.train() # 모델 학습 
    for epoch in range(EPOCHS): # EPOCHS 만큼 반복 
        correct = 0 # 초기화 
        for batch_idx, (X_batch, y_batch) in enumerate(loader_train): # loader_train 순회 
            var_X_batch = Variable(X_batch).float() # PyTorch Tensor의 Wrapper
            var_y_batch = Variable(y_batch)  # PyTorch Tensor의 Wrapper
            optimizer.zero_grad() # 그레디언트 초기화 
            output = model(var_X_batch)  # 모델 적용 
            loss = error(output, var_y_batch) # 손실
            loss.backward() # 손실 역전파 
            optimizer.step() # 역전파 단계에서 수집된 변화도로 매개변수를 조정
            predicted = torch.max(output.data, 1)[1]   #  return max_indices
            correct += (predicted == var_y_batch).sum() # correct count 
            if batch_idx % 50 == 0:
                print('에포크 : {} [{}/{} ({:.0f}%)]\t 손실함수 : {:.6f}\t Accuracy:{:.3f}%'\
                      .format(epoch, batch_idx*len(X_batch),len(loader_train),\
                        100.*batch_idx / len(loader_train),loss.data,correct*100./ (BATCH_SIZE*(batch_idx + 1))))


In [ ]:
def evaluate(model): # 모델 평가 
    correct = 0
    for test_imgs, test_labels in loader_test:
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        # 가장 높은 값을 가진 인덱스:  예측값
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("테스트 데이터 정확도: {:.3f}% ".format(float(correct) / (len(loader_test)*BATCH_SIZE)))

cnn = CNN()
evaluate(cnn)
fit(cnn, loader_train)
cnn.eval()
evaluate(cnn)
index = 10
data = X_test[index].view(-1, 1, 28, 28).float()
output = cnn(data)
print('{} 번째 학습데이터의 테스트 결과 : {}'.format(index,output))
_, predicted = torch.max(output, 1)
print('{} 번째 데이터의 예측측 : {}'.format(index, predicted.numpy()))
print('실제 레이블 : {}'.format(y_test[index]))

테스트 데이터 정확도: 0.103% 
에포크 : 0 [0/1875 (0%)]	 손실함수 : 15.513609	 Accuracy:9.375%
에포크 : 0 [1600/1875 (3%)]	 손실함수 : 1.649032	 Accuracy:19.792%
에포크 : 0 [3200/1875 (5%)]	 손실함수 : 1.346962	 Accuracy:37.005%
에포크 : 0 [4800/1875 (8%)]	 손실함수 : 0.957614	 Accuracy:47.268%
에포크 : 0 [6400/1875 (11%)]	 손실함수 : 0.667829	 Accuracy:53.980%
에포크 : 0 [8000/1875 (13%)]	 손실함수 : 0.498213	 Accuracy:58.503%
에포크 : 0 [9600/1875 (16%)]	 손실함수 : 0.299985	 Accuracy:62.791%
에포크 : 0 [11200/1875 (19%)]	 손실함수 : 0.707015	 Accuracy:65.856%
에포크 : 0 [12800/1875 (21%)]	 손실함수 : 0.520352	 Accuracy:68.259%
에포크 : 0 [14400/1875 (24%)]	 손실함수 : 0.568004	 Accuracy:70.295%
에포크 : 0 [16000/1875 (27%)]	 손실함수 : 0.401867	 Accuracy:72.000%
에포크 : 0 [17600/1875 (29%)]	 손실함수 : 0.311450	 Accuracy:73.440%
에포크 : 0 [19200/1875 (32%)]	 손실함수 : 0.504171	 Accuracy:74.532%
에포크 : 0 [20800/1875 (35%)]	 손실함수 : 0.346131	 Accuracy:75.715%
에포크 : 0 [22400/1875 (37%)]	 손실함수 : 0.264213	 Accuracy:76.716%
에포크 : 0 [24000/1875 (40%)]	 손실함수 : 0.493441	 Accuracy:77.671%
에